In [1]:
import numpy as np
import matplotlib.pyplot as plt
from jackknife import *
from scipy.optimize import curve_fit
from scipy.integrate import quad
from scipy.special import gamma
from corr_fits import *
from spectral_density import *
import mpmath

In [2]:
df = open("out/vector/data/correlators/run_vector_folded_ml_4.txt", "r")
jf = open("out/vector/data/correlators/run_vector_jack.txt", "r")

dat = df.readlines()

corr_jack = []

for line in dat:
    jack = np.zeros(25)
    for i in range(25):
        jack[i]  = float(jf.readline())
    corr_jack.append(Jackknife())
    corr_jack[-1].mean = float((line.split())[0])
    corr_jack[-1].jack = jack

corr, err_corr, cov_corr = Jackknife.to_lists(corr_jack)
        
df.close()
jf.close()

In [3]:
model = Double_exp_np()
par, _ = curve_fit(model.fit_f, np.arange(10, 35), corr[10:35], [0.001, 0.27, 0.004, 0.511], sigma=err_corr[10:35])
print(par)
cf = Corr_fits(model.f, model.der_list(), par, np.arange(10, 35), corr[10:35], cov_corr[10:35, 10:35])
cf.cov_par()

[0.00124701 0.273436   0.00434945 0.51144887]


array([[1.61325329e-08, 5.93670198e-07, 8.21919631e-08, 4.06303010e-06],
       [5.93670198e-07, 2.22936126e-05, 2.89971661e-06, 1.45740412e-04],
       [8.21919631e-08, 2.89971661e-06, 4.84419259e-07, 2.25178848e-05],
       [4.06303010e-06, 1.45740412e-04, 2.25178848e-05, 1.07426145e-03]])

In [4]:
x = np.arange(10, 35)
W = np.diag(1/err_corr[10:35])
J = np.zeros((len(par), len(x)))
for i in range(len(x)):
    J[:, i] = model.jac(x[i], par)

H_inv = np.linalg.inv(J @ W @ W @ J.T)
C = cov_corr[10:35, 10:35]
cov_par = H_inv @ J @ W @ W @ C @ W @ W @ J.T @ H_inv

In [5]:
x = np.arange(0, 48)
J = np.zeros((len(par), len(x)))
for i in range(len(x)):
    J[:, i] = model.jac(x[i], par)

np.sqrt(np.diagonal(J.T @ cov_par @ J))

array([8.15435907e-04, 4.19330562e-04, 2.11540696e-04, 1.04215346e-04,
       4.98225219e-05, 2.29019245e-05, 9.97850125e-06, 4.02432171e-06,
       1.44334852e-06, 4.56002026e-07, 2.32180698e-07, 2.18644726e-07,
       1.97438758e-07, 1.72779174e-07, 1.51807164e-07, 1.33699462e-07,
       1.17200893e-07, 1.02221312e-07, 8.90659758e-08, 7.78304838e-08,
       6.83028325e-08, 6.01147671e-08, 5.29098517e-08, 4.64310234e-08,
       4.05307343e-08, 3.51421666e-08, 3.02434105e-08, 2.58295914e-08,
       2.18960639e-08, 1.84306224e-08, 1.54115400e-08, 1.28087376e-08,
       1.05862528e-08, 8.70493065e-09, 7.12479712e-09, 5.80690178e-09,
       4.71460472e-09, 3.81437113e-09, 3.07617141e-09, 2.47358835e-09,
       1.98372123e-09, 1.58696038e-09, 1.26668832e-09, 1.00894822e-09,
       8.02107937e-10, 6.36537533e-10, 5.04311410e-10, 3.98940382e-10])